In [ ]:
!pip install earthengine-api
from google.colab import auth
auth.authenticate_user()
!earthengine authenticate


In [2]:
import ee
import unicodedata

# ==============================================================================
# 0) XÁC THỰC VÀ KHỞI TẠO
# ==============================================================================
PROJECT_ID = 'ambient-airlock-472009-u8'

try:
    ee.Initialize(project=PROJECT_ID)
    print("✅ Google Earth Engine đã được khởi tạo.")
except Exception:
    print("⚠️ Cần xác thực tài khoản...")
    ee.Authenticate()
    ee.Initialize(project=PROJECT_ID)
    print("✅ Google Earth Engine đã được khởi tạo sau khi xác thực.")

# ==============================================================================
# 1) CÀI ĐẶT ĐẦU VÀO CHUNG
# ==============================================================================
YEAR = 2025
start_date = ee.Date(f'{YEAR}-01-01')
end_date   = ee.Date(f'{YEAR}-12-31').advance(1, 'day')  # cover hết 31/12

gadm_districts = ee.FeatureCollection(
    'projects/ambient-airlock-472009-u8/assets/gadm41_VNM_2'
)

# Có thể bật lại toàn bộ 13 tỉnh khi mọi thứ chạy ổn
mekong_delta_provinces = [
    'Long An', 'Tiền Giang', 'Bến Tre', 'Trà Vinh', 'Vĩnh Long',
    'Đồng Tháp', 'An Giang',
    'Kiên Giang',
    'Hậu Giang',
    'Sóc Trăng', 'Bạc Liêu', 'Cà Mau', 'Cần Thơ'
]

# MODIS NDVI 16-day (MOD13Q1) toàn vùng Mekong (sẽ filter theo tỉnh sau)
modis_ndvi_collection = (
    ee.ImageCollection('MODIS/061/MOD13Q1')
    .filterDate(start_date, end_date)
    .filterBounds(gadm_districts)  # rộng, sau filter tỉnh
)

# Lấy projection NDVI (250m) – dùng cho scale reduceRegions, không reprojection
modis_sample = ee.Image(modis_ndvi_collection.first())
modis_proj = modis_sample.select('NDVI').projection()

# Dynamic World collection chung (sẽ filter theo tỉnh sau)
dw_col_global = (
    ee.ImageCollection('GOOGLE/DYNAMICWORLD/V1')
    .filterDate(start_date, end_date)
    .filterBounds(gadm_districts)
)

dw_sample = ee.Image(dw_col_global.first())
dw_proj = dw_sample.select('label').projection()

# ==============================================================================
# 2) HÀM TIỆN ÍCH
# ==============================================================================

def slugify_vn(name):
    """Đổi tên tỉnh có dấu thành dạng không dấu & dùng '_' cho file name."""
    s = unicodedata.normalize('NFKD', name).encode('ascii', 'ignore').decode('utf-8')
    s = s.replace(' ', '_')
    return s

def prepare_and_mask_ndvi_16day(image, cropland_mask_10m):
    """
    Chuẩn hóa NDVI, QA mask, rồi mask bằng cropland mask 10m.
    Không dùng fraction 250m để tránh lỗi reprojection / reduceResolution.
    """
    ndvi = image.select('NDVI').multiply(0.0001).rename('NDVI')

    qa_band = image.select('SummaryQA')
    quality_mask = qa_band.lt(2)

    clean_ndvi = (
        ndvi
        .updateMask(quality_mask)
        .updateMask(cropland_mask_10m)   # mask trực tiếp bằng DW cropland 10m
    )

    return clean_ndvi.copyProperties(image, ['system:time_start'])

def reduce_by_district(image, district_fc):
    """Tính mean/median/stdDev NDVI theo huyện cho 1 ảnh 16 ngày."""
    reducer = (
        ee.Reducer.mean()
        .combine(ee.Reducer.median(), '', True)
        .combine(ee.Reducer.stdDev(), '', True)
    )

    stats_fc = image.reduceRegions(
        collection=district_fc,
        reducer=reducer,
        scale=250,     # scale MOD13Q1
        tileScale=2    # nếu còn lỗi memory có thể tăng 4 hoặc 8
    )

    start_time = ee.Date(image.get('system:time_start'))
    end_time   = start_time.advance(15, 'day')  # MOD13Q1 = 16-day composite

    def add_properties(feature):
        return feature.set({
            'start_date': start_time.format('YYYY-MM-dd'),
            'end_date':   end_time.format('YYYY-MM-dd')
        })

    # Chỉ giữ huyện có NDVI (mean không null)
    return stats_fc.filter(ee.Filter.notNull(['mean'])).map(add_properties)

# ==============================================================================
# 3) LOOP QUA TỪNG TỈNH & TẠO TASK EXPORT
# ==============================================================================
for province_name in mekong_delta_provinces:
    print(f"\n🔹 Đang xử lý tỉnh: {province_name}")

    # --- 3.1) Lọc huyện của tỉnh này ---
    province_districts = gadm_districts.filter(
        ee.Filter.eq('NAME_1', province_name)
    )
    province_geometry = province_districts.geometry()

    # --- 3.2) Lọc NDVI theo tỉnh ---
    modis_ndvi_prov = modis_ndvi_collection.filterBounds(province_geometry)

    # --- 3.3) DYNAMIC WORLD theo tỉnh ---
    dw_col_prov = dw_col_global.filterBounds(province_geometry)

    # MODE theo thời gian (chọn lớp cropland ổn định trong năm)
    dw_label_mode_prov = dw_col_prov.select('label').reduce(ee.Reducer.mode())

    # Cropland 10m nhị phân (0/1), đã cắt theo tỉnh
    cropland_mask_prov = (
        dw_label_mode_prov
        .eq(4)                     # class 4 = cropland
        .rename('cropland_10m')
        .clip(province_geometry)
    )

    # --- 3.4) NDVI đã mask theo DW cho tỉnh này ---
    def mapper_ndvi(img):
        return prepare_and_mask_ndvi_16day(img, cropland_mask_prov)

    processed_ndvi_prov = modis_ndvi_prov.map(mapper_ndvi)

    # --- 3.5) Thống kê theo huyện cho từng ảnh 16 ngày ---
    def mapper_reduce(img):
        return reduce_by_district(img, province_districts)

    final_fc_prov = ee.FeatureCollection(
        processed_ndvi_prov.map(mapper_reduce)
    ).flatten()

    # --- 3.6) EXPORT TỪNG TỈNH ---
    province_slug = slugify_vn(province_name)
    output_filename = (
        f'NDVI_MODIS16DAY_{YEAR}_{province_slug}_DW_CropMask10m'
    )

    output_columns = [
        'NAME_1', 'GID_1', 'NAME_2', 'GID_2',
        'start_date', 'end_date',
        'mean', 'median', 'stdDev'
    ]

    task = ee.batch.Export.table.toDrive(
        collection=final_fc_prov,
        description=output_filename,
        folder='GEE_NDVI_Exports',  # nhớ tạo hoặc đổi tên folder nếu cần
        fileNamePrefix=output_filename,
        fileFormat='CSV',
        selectors=output_columns
    )
    task.start()
    print(f"🚀 Đã start task xuất dữ liệu cho tỉnh {province_name}: {output_filename}")

print("\n✅ Đã tạo xong tất cả task export.")


✅ Google Earth Engine đã được khởi tạo.
